In [1]:
# USE_SAME_FEW_SHOTS=True

In [2]:
num_shots = 5

In [3]:
experiment_name = "2024.05.17-02_Zephyr-7b-beta_4bit_5-few-shot_SEMANTIC-SIMILARITY"

In [4]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"

input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx"

top7_train_data_sim_to_each_test_data_filename = base_path + "/input/top7_train_data_sim_to_each_test_data_2023.12.11.pickle"
# top7_train_data_sim_to_each_test_data_filename = base_path + "/input/top7_train_data_sim_to_each_test_data_but_with_different_classes_with_2023.12.11_input_data.pickle"

output_data_path = base_path + "/../Mistral/data/output/" + experiment_name
log_filename = base_path + "/../Mistral/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [5]:
!pip install -q accelerate bitsandbytes rich transformers --progress-bar off

In [6]:
import torch
from transformers import pipeline
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import BitsAndBytesConfig

In [7]:
import pickle
import pandas as pd

In [8]:
# if USE_SAME_FEW_SHOTS==True:
#     data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])
#     data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
#         (data_sel_4_few_shot_examples.text_id==5614) |
#         (data_sel_4_few_shot_examples.text_id==2089) |
#         (data_sel_4_few_shot_examples.text_id==5665) |
#         (data_sel_4_few_shot_examples.text_id==370)  |
#         (data_sel_4_few_shot_examples.text_id==1422) |
#         (data_sel_4_few_shot_examples.text_id==4329) |
#         (data_sel_4_few_shot_examples.text_id==4929)
#     ]
# else:
#     data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0']).sample(3)

# data_sel_4_few_shot_examples

In [9]:
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"SDG-"+str(row['sdg'])+'\"\n\n'

#     few_shots += input_text
#     few_shots += output_text

# print(few_shots)

In [10]:
with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [11]:
data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])

In [12]:
pretrained_model_name_or_path = "HuggingFaceH4/zephyr-7b-beta"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=pretrained_model_name_or_path,
    quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=pretrained_model_name_or_path
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [13]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

In [14]:
# @title
import pandas as pd
data = pd.read_excel(input_data_path)

## Run the model

In [15]:
def generate_query_prompt(query: str) -> str:
  """Generate LLM prompt for user query."""
  prompt = (
      "<|system|>\n"
      "You are a text classifier.</s>\n"
      "<|user|>\n"
      f"{query}.</s>\n"
      "<|assistant|>\n"
  )
  return prompt

In [16]:
data_sel = data

In [17]:
text_id = data_sel.iloc[0].text_id
text = data_sel.iloc[0].text.replace('\n','').replace('{','(').replace('}',')')
sdg_true = data_sel.iloc[0].sdg

print("EXPECTED_SDG:", sdg_true, "\nTEXT:", text.replace('\n',''),'\n\n')

few_shots_list = top7_train_data_sim_to_each_test_data[text_id][:num_shots]
few_shots = ""
for train_data_id in few_shots_list:
  train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]
  input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
  output_text = "Output: \"SDG-"+str(train_row.sdg.values[0])+'\"\n\n'
  few_shots += input_text
  few_shots += output_text
print(few_shots)

template = f"""Classify the following input text within triple quotes according to the following Sustainable Development Goals (SDGs) dictionary of labels:

"SDG-1": "End poverty in all its forms everywhere."
"SDG-2": "End hunger, achieve food security and improved nutrition and promote sustainable agriculture."
"SDG-3": "Ensure healthy lives and promote well-being for all at all ages."
"SDG-4": "Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all."
"SDG-5": "Achieve gender equality and empower all women and girls."
"SDG-6": "Ensure availability and sustainable management of water and sanitation for all."
"SDG-7": "Ensure access to affordable, reliable, sustainable and modern energy for all."
"SDG-8": "Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all."
"SDG-9": "Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation."
"SDG-10": "Reduce inequality within and among countries."
"SDG-11": "Make cities and human settlements inclusive, safe, resilient and sustainable."
"SDG-12": "Ensure sustainable consumption and production patterns."
"SDG-13": "Take urgent action to combat climate change and its impacts."
"SDG-14": "Conserve and sustainably use the oceans, seas and marine resources for sustainable development."
"SDG-15": "Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss."
"SDG-16": "Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels."
"SDG-0": "Other."

Choose ONLY ONE label for each input text.

DO NOT include the input text in your answer.

The following are """+str(num_shots)+f""" examples of expected classification given an input text:

"""+few_shots+f"""

The input text is:
'''
{text}
'''
"""

prompt = generate_query_prompt(query=template)
print(prompt)

EXPECTED_SDG: 0 
TEXT: L train resumes service after derailing from track in Canarsie, Brooklyn  WABC-TVService restored after subway train derails in Brooklyn: officials  PIX11 New York NewsNo injuries in Brooklyn subway derailment  New York Daily NewsView Full Coverage on Google News. 


Input text: "NYC crime: Woman stabbed on A train in Bed-Stuy; No arrests made  WABC-TVPolice: Man hit in head with gun, shot in leg on subway in Queens  CBS New YorkNYC Crime: Teen stabbed in hip on subway train in Brooklyn; No arrests made  WABC-TVNYPD: Woman attacked on train; another subway stabbing leaves teen hurt  NBC New YorkWoman stabbed on subway platform in Brooklyn following stabbing, shooting in transit Sunday  CBS New YorkView Full Coverage on Google News."
Output: "SDG-0"

Input text: "NYC crime: 3 violent attacks on subways in span of 8 hours in Brooklyn, Queens; No arrests made  WABC-TVView Full Coverage on Google News."
Output: "SDG-0"

Input text: "One dies after motorcycle crash in

In [18]:
output = pipe(text_inputs=prompt, max_new_tokens=64, temperature=0.0, return_full_text=False)
output

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


[{'generated_text': 'Output: "SDG-0"'}]

In [19]:
data_sel = data #.iloc[162:]

In [ ]:
results = []

for index, row in data_sel.iterrows():

  cnt = index

  text_id = row['text_id']
  text = row["text"].replace('\n','').replace('{','(').replace('}',')')
  sdg_true = row["sdg"]

  file = open(log_filename, 'a+', encoding='utf-8')

  print("\n\n##################################################")
  file.write("\n\n##################################################\n")

  print("EXPECTED_SDG:", row["sdg"], "TEXT:", row["text"].replace('\n',''),'\n\n')
  file.write(f"EXPECTED_SDG: {row['sdg']} TEXT: " + row['text'].replace('\n','') + "\n\n\n")

  # INIZIO AGGIUNTA PER SEMANTIC SIMILARITY
  few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:num_shots]
  few_shots = ""
  for train_data_id in few_shots_list:
    train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]
    input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
    output_text = "Output: \"SDG-"+str(train_row.sdg.values[0])+'\"\n\n'
    few_shots += input_text
    few_shots += output_text
  print(few_shots)
  # FINE AGGIUNTA PER SEMANTIC SIMILARITY

  template = f"""Classify the following input text within triple quotes according to the following Sustainable Development Goals (SDGs) dictionary of labels:

"SDG-1": "End poverty in all its forms everywhere."
"SDG-2": "End hunger, achieve food security and improved nutrition and promote sustainable agriculture."
"SDG-3": "Ensure healthy lives and promote well-being for all at all ages."
"SDG-4": "Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all."
"SDG-5": "Achieve gender equality and empower all women and girls."
"SDG-6": "Ensure availability and sustainable management of water and sanitation for all."
"SDG-7": "Ensure access to affordable, reliable, sustainable and modern energy for all."
"SDG-8": "Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all."
"SDG-9": "Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation."
"SDG-10": "Reduce inequality within and among countries."
"SDG-11": "Make cities and human settlements inclusive, safe, resilient and sustainable."
"SDG-12": "Ensure sustainable consumption and production patterns."
"SDG-13": "Take urgent action to combat climate change and its impacts."
"SDG-14": "Conserve and sustainably use the oceans, seas and marine resources for sustainable development."
"SDG-15": "Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss."
"SDG-16": "Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels."
"SDG-0": "Other."

Choose ONLY ONE label for each input text.

DO NOT include the input text in your answer.

The following are """+str(num_shots)+f""" examples of expected classification given an input text:

"""+few_shots+f"""

The input text is:
'''
{text}
'''
"""

  prompt = generate_query_prompt(query=template)

  output = pipe(text_inputs=prompt, max_new_tokens=1024, temperature=0.0, return_full_text=False)

  result_extended = (cnt, text_id, sdg_true, output)

  print(result_extended)
  file.write(str(result_extended)+'\n')

  with open(output_data_path+f'/sdg_classification_{cnt}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)

  file.close()




##################################################
EXPECTED_SDG: 0 TEXT: L train resumes service after derailing from track in Canarsie, Brooklyn  WABC-TVService restored after subway train derails in Brooklyn: officials  PIX11 New York NewsNo injuries in Brooklyn subway derailment  New York Daily NewsView Full Coverage on Google News. 


Input text: "NYC crime: Woman stabbed on A train in Bed-Stuy; No arrests made  WABC-TVPolice: Man hit in head with gun, shot in leg on subway in Queens  CBS New YorkNYC Crime: Teen stabbed in hip on subway train in Brooklyn; No arrests made  WABC-TVNYPD: Woman attacked on train; another subway stabbing leaves teen hurt  NBC New YorkWoman stabbed on subway platform in Brooklyn following stabbing, shooting in transit Sunday  CBS New YorkView Full Coverage on Google News."
Output: "SDG-0"

Input text: "NYC crime: 3 violent attacks on subways in span of 8 hours in Brooklyn, Queens; No arrests made  WABC-TVView Full Coverage on Google News."
Output: "SDG

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Streaming output truncated to the last 5000 lines.
Output: "SDG-5"

Input text: "Women’s rising levels of educational attainment - and the associated nanowing of the gender gap in education - are one of the main drivers of change, with major implications for gender equality, the formation and/or dissolution of partnerships and childbearing decisions (van Bavel, 2012). Educational attainment paid off for men both in the labour market (giving them access to higher salaries) and the marriage market (making them more attractive partners) (Becker, 1991). The traditional marriage pattern was also compatible with the gender imbalance in higher education in favour of men."
Output: "SDG-5"

Input text: "For this reason, a range of indicators are selected that capture gender inequality at different stages of the life cycle. Lastly, the chapter focuses on the indicators that have been shown to matter for the overall development of the societies (e.g. World Bank, 2011). For this reason, not all ke

In [ ]:
data_sel = data #.iloc[:3]

In [ ]:
import glob

files = glob.glob(output_data_path + "/*.pkl")

results = []
for file in files:
    res = pd.read_pickle(file)

    # try:
    #     if len(res[3]['choices'])>1:
    #         raise Exception('ERROR: Multilabel detected!')
    #         break
    #     sdg_pred = res[3]['choices'][0]['message']['content']
    #     res4df = (res[0], res[1], res[2], sdg_pred)
    #     print(res4df)
    #     tuple_list.append(res4df)
    # except:
    #     print("#################################################################")
    #     print(res)
    #     print("#################################################################")

    # print(res)
    results.append(res)
    # break

len(results)

In [ ]:
results_processed = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'result'])
results_processed.head(3)

In [ ]:
results_processed.iloc[0].result

In [ ]:
#results_processed.iloc[0].result.split("Answer with label only usign the format:---LABEL---")[1]
# results_processed['result_short'] = results_processed['result'].apply(lambda x: x.split("Answer with label only usign the format:---LABEL---")[1])

In [ ]:
# results_processed['result']

In [ ]:
# results_processed['label'] = results_processed['result'].str.extract(r'SDG-(\d+):')
# results_processed['label'] = results_processed['result_short'].str.extract(r'SDG (\d+)')

# results_processed['label'] = results_processed['result'].apply(
#     lambda x: pd.Series(x[0]['generated_text']).str.extract(r'SDG-(\d+)')[0]
# )

results_processed['label'] = results_processed['result'].apply(
    lambda x: pd.Series(x[0]['generated_text']).str.extract(r'SDG-(\d+)')[0]
)
# results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
tmp_df_1 = results_processed[~results_processed.label.isna()]

results_processed['label'] = results_processed['result'].apply(
    lambda x: pd.Series(x[0]['generated_text']).str.extract(r'SDG (\d+)')[0]
)
# results_processed['label'] = results_processed['result'].str.extract(r'SDG-(\d+)')
tmp_df_2 = results_processed[~results_processed.label.isna()]

results_processed = pd.concat([tmp_df_1,tmp_df_2])
results_processed

# results_processed.drop_duplicates()

results_processed.head(3)

In [ ]:
results_processed = results_processed.drop_duplicates(['cnt', 'text_id', 'sdg_true', 'label'])

In [ ]:
results_processed.shape

In [ ]:
# Individua i duplicati basati sulla colonna 'text_id'
# duplicate_text_ids = results_processed[results_processed.duplicated('text_id', keep=False)]
# duplicate_text_ids.sort_values(['text_id'])

In [ ]:
final = data_sel.merge(results_processed, how='left', left_on='text_id', right_on='text_id')
final.head(3)

In [ ]:
final.shape

In [ ]:
data_sel.shape[0]

In [ ]:
text_ids_with_errors = final[final.label.isna()].text_id.to_list()
list(set(text_ids_with_errors))

In [ ]:
final_1  = final[~final.label.isna()]

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final_1.sdg
y_pred = final_1.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

In [ ]:
# classification_report_filename = output_data_path+f'/{experiment_name}_classification_report.xlsx'
# classification_report_filename

In [ ]:
# report=classification_report(y_true, y_pred, output_dict=True)
# report_df = pd.DataFrame(report).transpose()
# report_df.to_excel(classification_report_filename)

In [ ]:
# test = pd.read_excel(classification_report_filename)
# test

In [ ]:
results_processed_merged = final

In [ ]:
results_processed_merged.shape

In [ ]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
sdg_list = [str(x) for x in range(17)]
# sdg_list = [x for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['label']) or row['label'] not in sdg_list:
#         new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'label'] = 0  # new_value

In [ ]:
results_processed_merged.head(3)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg
y_pred = results_processed_merged.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

In [ ]:
output_datapath = output_data_path

In [ ]:
classification_report_filename = output_datapath+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
report_df.to_excel(classification_report_filename)

In [ ]:
report_df

In [ ]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [ ]:
print(precision, recall, accuracy, f1_score)

In [ ]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))